In [1]:
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
%matplotlib inline

df=pd.read_excel('Election_Data.xlsx',sheet_name='Election_Dataset_Two Classes')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)
df =pd.get_dummies(df, columns=['gender','vote'],drop_first=True)
df

,age,economic.cond.national,economic.cond.household,Blair,Hague,Europe,political.knowledge,gender_male,vote_Labour
0,43,3,3,4,1,2,2,0,1
1,36,4,4,4,4,5,2,1,1
2,35,4,4,5,2,3,2,1,1
3,24,4,2,2,1,4,0,0,1
4,41,2,2,1,1,6,2,1,1
...,...,...,...,...,...,...,...,...,...
1512,67,5,3,2,4,11,3,1,0
1513,73,2,2,4,4,8,2,1,0
1514,37,3,3,5,4,2,2,1,1
1515,61,3,3,1,4,11,2,1,0


In [2]:
X = df.drop("vote_Labour" , axis=1)
y = df['vote_Labour']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=1, stratify=y)

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
grid={ 'criterion':['gini'],  # 'entropy'
    'splitter':['random'],   # 'best'
    'max_depth':[7],   # 4,5,6,8,9,10
    'min_samples_split':[2],   # 3,4,5,10
    'min_samples_leaf':[7],   # 4,5,6,8,9,10,11,12,13
    'max_features':[6]  # 5,7,8
    }   
model = DecisionTreeClassifier(random_state=1)
grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 3,n_jobs=-1,scoring='f1')

grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_
model=clf.fit(X_train,y_train)

cart2 = grid_search.best_estimator_
grid={ 'n_estimators':[200],   # 10,20,50,5,6,7,8,9,30,100,400,500,250,300,350,100,150
    'max_samples':[0.8],    # 1.0,0.7,0.5
    'max_features':[1.0],   # 0.5,0.7,0.3
    'bootstrap':[True],   # False
    'bootstrap_features':[False],   # True
    'oob_score':[False],   # True
    'warm_start':[False],   # True
    'n_jobs':[-1],
    'verbose':[0]}   # 1
model = BaggingClassifier(base_estimator=cart2,random_state=1)
grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 3,n_jobs=-1,scoring='accuracy')

grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_
model=clf.fit(X_train,y_train)

In [5]:
model.score(X_train, y_train)

0.8548539114043355

In [6]:
model.score(X_test, y_test)

0.8574561403508771

In [7]:
pred_train_prob = model.predict_proba(X_train)
pred_test_prob = model.predict_proba(X_test)
pred_train_prob

array([[0.08067069, 0.91932931],
       [0.00717657, 0.99282343],
       [0.53832583, 0.46167417],
       ...,
       [0.02673066, 0.97326934],
       [0.67927228, 0.32072772],
       [0.69281251, 0.30718749]])

In [8]:
pred_train_prob = pred_train_prob[:,1]

In [9]:
from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score


def GetCustomData(custom_prob):
    custom_cutoff_data=[]

    for i in range(0,len(pred_train_prob)):  #2333
        if pred_train_prob[i] >custom_prob:
            a= 1
        else:
            a=0
        custom_cutoff_data.append(a)
    return custom_cutoff_data

In [10]:
thresh=[]
acc=[]
f1=[]
rec=[]
pre=[]

In [11]:
for j in np.arange(0.1,1,0.1):
    result = GetCustomData(j)
    thresh.append(j)
    acc.append(accuracy_score(y_train,result))
    f1.append(f1_score(y_train,result))
    rec.append(recall_score(y_train,result))
    pre.append(precision_score(y_train,result))


In [12]:
dat = pd.DataFrame({'Threshold': thresh,
             'Accuracy': acc,
             'f1': f1,
             'Recall': rec,
             'Precision':pre})
dat

,Threshold,Accuracy,f1,Recall,Precision
0,0.1,0.700283,0.822940,1.000000,0.699149
1,0.2,0.770971,0.857645,0.990528,0.756198
2,0.3,0.793591,0.869097,0.983762,0.778373
3,0.4,0.836946,0.892077,0.967524,0.827546
4,0.5,0.854854,0.897742,0.914750,0.881356
5,0.6,0.852969,0.891817,0.870095,0.914651
6,0.7,0.822809,0.862573,0.798376,0.937997
7,0.8,0.750236,0.788169,0.667118,0.962891
8,0.9,0.659755,0.680248,0.519621,0.984615


In [13]:
dat[dat['f1']==dat['f1'].max()]

,Threshold,Accuracy,f1,Recall,Precision
4,0.5,0.854854,0.897742,0.91475,0.881356


In [14]:
dat[dat['Accuracy']==dat['Accuracy'].max()]

,Threshold,Accuracy,f1,Recall,Precision
4,0.5,0.854854,0.897742,0.91475,0.881356


In [50]:
from sklearn.ensemble import RandomForestClassifier
grid={'n_estimators':[100],   # 50,150,200,300,500
    'criterion':['gini'],   # 'entropy'
    'max_depth':[5],   # 6,7,8,9
    'min_samples_split':[2],  #5,10,20
    'min_samples_leaf':[1],   #3,5,7,10
    'max_features':[2],   # 3,4,5,6
    'bootstrap':[True],  # False
    'oob_score':[False],  # True
    'warm_start':[False],  # True
    'ccp_alpha':[0.0,]   # 0.2,0.4
}
model = RandomForestClassifier(random_state=1)
grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 3,n_jobs=-1,scoring='f1')

grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_
rf=clf.fit(X_train,y_train)

grid={ 'n_estimators':[200],   # 10,20,50,5,6,7,8,9,30,100,400,500,250,300,350,100,150
    'max_samples':[0.4],    # 1.0,0.7,0.5
    'max_features':[1.0],   # 0.5,0.7,0.3
    'bootstrap':[True],   # False
    'bootstrap_features':[False],   # True
    'oob_score':[False],   # True
    'warm_start':[False],   # True
    'n_jobs':[-1],
    'verbose':[0]}   # 1
model = BaggingClassifier(base_estimator=rf,random_state=1)

grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 3,n_jobs=-1,scoring='accuracy')

In [51]:
grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_
model=clf.fit(X_train,y_train)

In [52]:
model.score(X_train, y_train)

0.8520263901979265

In [53]:
model.score(X_test, y_test)

0.8530701754385965

In [54]:
pred_train_prob = model.predict_proba(X_train)
pred_test_prob = model.predict_proba(X_test)
pred_train_prob

array([[0.16551608, 0.83448392],
       [0.03946336, 0.96053664],
       [0.53142311, 0.46857689],
       ...,
       [0.06114707, 0.93885293],
       [0.58383813, 0.41616187],
       [0.591566  , 0.408434  ]])

In [55]:
pred_train_prob = pred_train_prob[:,1]

In [56]:
from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score


def GetCustomData(custom_prob):
    custom_cutoff_data=[]

    for i in range(0,len(pred_train_prob)):  #2333
        if pred_train_prob[i] >custom_prob:
            a= 1
        else:
            a=0
        custom_cutoff_data.append(a)
    return custom_cutoff_data

In [57]:
thresh=[]
acc=[]
f1=[]
rec=[]
pre=[]

In [58]:
for j in np.arange(0.1,1,0.1):
    result = GetCustomData(j)
    thresh.append(j)
    acc.append(accuracy_score(y_train,result))
    f1.append(f1_score(y_train,result))
    rec.append(recall_score(y_train,result))
    pre.append(precision_score(y_train,result))


In [59]:
dat = pd.DataFrame({'Threshold': thresh,
             'Accuracy': acc,
             'f1': f1,
             'Recall': rec,
             'Precision':pre})
dat

,Threshold,Accuracy,f1,Recall,Precision
0,0.1,0.696513,0.821111,1.000000,0.696513
1,0.2,0.711593,0.828475,1.000000,0.707177
2,0.3,0.770971,0.857478,0.989175,0.756729
3,0.4,0.816211,0.881315,0.979702,0.800885
4,0.5,0.852026,0.898644,0.941813,0.859259
5,0.6,0.855796,0.894264,0.875507,0.913842
6,0.7,0.824694,0.863836,0.798376,0.940989
7,0.8,0.719133,0.754530,0.619756,0.964211
8,0.9,0.561734,0.545455,0.377537,0.982394


In [60]:
dat[dat['f1']==dat['f1'].max()]

,Threshold,Accuracy,f1,Recall,Precision
4,0.5,0.852026,0.898644,0.941813,0.859259


In [61]:
dat[dat['Accuracy']==dat['Accuracy'].max()]

,Threshold,Accuracy,f1,Recall,Precision
5,0.6,0.855796,0.894264,0.875507,0.913842
